In [78]:
import ssl
import requests
from bs4 import BeautifulSoup
import difflib
import csv
#Preparing context for handeling SSL certificate errors
ctx=ssl.create_default_context()
ctx.check_hostname=False
ctx.verify_mode=ssl.CERT_NONE

In [79]:
with open('header_col.csv') as filer:
    reader=csv.reader(filer)
    headers=list()
    for item in reader:
        headers.append(item)

headers=headers[0]
print(headers)

['Directed by', 'Produced by', 'Screenplay by', 'Based on', 'Starring', 'Music by', 'Cinematography', 'Edited by', 'Productioncompanies ', 'Distributed by', 'Release date', 'Running time', 'Country', 'Language', 'Budget', 'Box office']


In [80]:
url='https://en.wikipedia.org/wiki/List_of_Academy_Award-winning_films'
base='https://en.wikipedia.org'

page=requests.get(url=url)

soup=BeautifulSoup(page.content,'html.parser')

In [81]:
number_of_rows = len(soup.find_all('tr'))
print(number_of_rows)

1344


In [82]:
print(difflib.get_close_matches('directed by',headers,cutoff=0.90))

['Directed by']


In [83]:
column_selection=dict()
for value in headers:
    column_selection[value]=0
print(column_selection)

{'Directed by': 0, 'Produced by': 0, 'Screenplay by': 0, 'Based on': 0, 'Starring': 0, 'Music by': 0, 'Cinematography': 0, 'Edited by': 0, 'Productioncompanies ': 0, 'Distributed by': 0, 'Release date': 0, 'Running time': 0, 'Country': 0, 'Language': 0, 'Budget': 0, 'Box office': 0}


In [84]:
print(column_selection.keys())

dict_keys(['Directed by', 'Produced by', 'Screenplay by', 'Based on', 'Starring', 'Music by', 'Cinematography', 'Edited by', 'Productioncompanies ', 'Distributed by', 'Release date', 'Running time', 'Country', 'Language', 'Budget', 'Box office'])


In [85]:
def lowupfind(word):
    Length=len(word)
    for i in range(Length-1):
        if i==0:
            continue
        if word[i].islower() and word[i+1].isupper():
            word=word[:i+1]+' '+word[i+1:]
    return word

In [86]:
print(lowupfind('OlivierRenée'))

Olivier Renée


In [87]:
#Selecting the first movie as reference
import time
movie_dataset=list()
for line in range(number_of_rows):
    '''
    if line==50:
        break
    '''
    data={}
    header='https://en.wikipedia.org'
    try:
        movie_list=soup.find_all('i')[line].find('a')
        if movie_list is None:   #This is called only for extra useless <i> tag
            raise Exception(line)
    except Exception as e:
        print(e)
        continue
    data['Name']=movie_list.text #Assigning movie names
    response=requests.get(header+movie_list['href'])
    item_movie=BeautifulSoup(response.content,'html.parser')
    row_table=item_movie.find_all('tr')
        #Find director
    for value in row_table:
        try:
            lb=value.find_all('th',{'class':'infobox-label'})[0]
            match_find=difflib.get_close_matches(lb.text,headers,cutoff=0.8) #Matching header name with the coloumn data we want
            if len(match_find)>0:
                matched_col=match_find[0]
                info=value.find_all('td',{'class':'infobox-data'})[0] #Finding all data under the header
                
                temp=info.text.split('[')[0]
                #data[matched_col]=temp
                #'''
                temp=temp.replace('\xa0',' ')
                temp=lowupfind(temp)
                if temp[0]=='\n' and temp[-1]=='\n': #For removing all 
                    data[matched_col]=temp[1:-1].replace('\n',', ')
                elif temp[0]=='\n' and temp[-1]!='\n':
                    data[matched_col]=temp[1:].replace('\n',', ')
                else:
                    data[matched_col]=temp
                #'''
                #print('--------')
                #data[matched_col]
                #print(lb.text,matched_col)
                #time.sleep(1)
            else:
                pass
           
        except:
            pass
    movie_dataset.append(data)


62
208
212
224
239
280
282
285
305
329
335
339
348
358
393
407
420
424
463
492
505
510
527
541
556
566
588
610
627
639
654
656
665
668
686
702
707
710
725
739
756
772
787
805
808
831
886
889
925
968
996
1003
1012
1017
1038
1044
1095
1102
1118
1164
1173
1176
1227
1237
1249
1258
1260
1265
1298
1326


In [90]:
val=5
movie_dataset[val]

{'Name': 'Sound of Metal',
 'Directed by': 'Darius Marder',
 'Produced by': 'Bill Benz, Kathy Benz, Bert Hamelinck, Sacha Ben Harroche',
 'Screenplay by': 'Darius Marder, Abraham Marder',
 'Starring': 'Riz Ahmed, Olivia Cooke, Paul Raci, Lauren Ridloff, Mathieu Amalric',
 'Music by': 'Abraham Marder, Nicolas Becker',
 'Cinematography': 'Daniël Bouquet',
 'Edited by': 'Mikkel E. G. Nielsen',
 'Productioncompanies ': 'Caviar, Ward Four, Flat 7 Productions',
 'Distributed by': 'Amazon Studios',
 'Release date': 'September 6, 2019 (2019-09-06) (TIFF), November 20, 2020 (2020-11-20) (United States)',
 'Running time': '120 minutes',
 'Country': 'United States',
 'Language': 'American Sign Language, English',
 'Budget': '$5.4 million',
 'Box office': '$176,177'}

In [93]:
import pandas as pd 
movie_data_frame=pd.DataFrame(movie_dataset)
movie_data_frame[0:50]

,Name,Directed by,Produced by,Screenplay by,Based on,Starring,Music by,Cinematography,Edited by,Productioncompanies,Distributed by,Release date,Running time,Country,Language,Budget,Box office
0,Nomadland,Chloé Zhao,"Frances Mc Dormand, Peter Spears, Mollye Asher...",Chloé Zhao,Nomadland: Surviving America in the Twenty-Fir...,"Frances Mc Dormand, David Strathairn, Linda Ma...",Ludovico Einaudi,Joshua James Richards,Chloé Zhao,"Highwayman Films, Hear/Say Productions, Cor Co...",Searchlight Pictures,"September 11, 2020 (2020-09-11) (Venice), Febr...",108 minutes,United States,English,$5 million,$20.7 million
1,The Father,Florian Zeller,David Parfitt Jean-Louis Livi Philippe Carcass...,Florian Zeller Christopher Hampton,Le Pèreby Florian Zeller,"Anthony Hopkins, Olivia Colman, Mark Gatiss, I...",Ludovico Einaudi,Ben Smithard,Yorgos Lamprinos,"F comme Film, Trademark Films, Cine@, AG Studi...","Lionsgate(United Kingdom), UGC Distribution(Fr...","27 January 2020 (2020-01-27) (Sundance), 26 Ma...",97 minutes,NaN,English,$6 million,$14.1 million
2,Judas and the Black Messiah,Shaka King,"Charles D. King, Ryan Coogler, Shaka King","Will Berson, Shaka King",NaN,"Daniel Kaluuya, Lakeith Stanfield, Jesse Plemo...","Mark Isham, Craig Harris",Sean Bobbitt,Kristan Sprague,"MACRO, Participant, Bron Creative, Proximity",Warner Bros. Pictures,"February 12, 2021",126 minutes,United States,English,$26 million,$6.5 million
3,Minari,Lee Isaac Chung,"Dede Gardner, Jeremy Kleiner, Christina Oh",NaN,NaN,"Steven Yeun, Han Ye-ri, Alan Kim, Noel Kate Ch...",Emile Mosseri,Lachlan Milne,Harry Yoon,Plan B Entertainment,A24,"February 12, 2021",115 minutes,United States,Korean,$2 million,$15 million
4,Mank,David Fincher,"Ceán Chaffin, Eric Roth, Douglas Urbanski",Jack Fincher,NaN,"Gary Oldman, Amanda Seyfried, Lily Collins, Ar...","Trent Reznor, Atticus Ross",Erik Messerschmidt,Kirk Baxter,"Netflix International Pictures, Flying Studio",Netflix,"November 13, 2020 (2020-11-13) (United States)",131 minutes,United States,English,$25 million,"$122,252"
5,Sound of Metal,Darius Marder,"Bill Benz, Kathy Benz, Bert Hamelinck, Sacha B...","Darius Marder, Abraham Marder",NaN,"Riz Ahmed, Olivia Cooke, Paul Raci, Lauren Rid...","Abraham Marder, Nicolas Becker",Daniël Bouquet,Mikkel E. G. Nielsen,"Caviar, Ward Four, Flat 7 Productions",Amazon Studios,"September 6, 2019 (2019-09-06) (TIFF), Novembe...",120 minutes,United States,"American Sign Language, English",$5.4 million,"$176,177"
6,Ma Rainey's Black Bottom,George C. Wolfe,"Denzel Washington, Todd Black, Dany Wolf",Ruben Santiago-Hudson,Ma Rainey's Black Bottomby August Wilson,"Viola Davis, Chadwick Boseman, Glynn Turman, C...",Branford Marsalis,Tobias A. Schliessler,Andrew Mondshein,"Escape Artists, Mundy Lane Entertainment",Netflix,"November 25, 2020 (2020-11-25) (United States)",94 minutes,United States,English,$20–22.5 million,NaN
7,Promising Young Woman,Emerald Fennell,"Margot Robbie, Josey Mc Namara, Tom Ackerley, ...",NaN,NaN,"Carey Mulligan, Bo Burnham, Alison Brie, Clanc...",Anthony Willis,Benjamin Kračun,Frédéric Thoraval,"Film Nation Entertainment, Lucky Chap Entertai...",Focus Features,"January 25, 2020 (2020-01-25) (Sundance), Dece...",113 minutes,NaN,English,$5–10 million,$16.3 million
8,Tenet,Christopher Nolan,"Emma Thomas, Christopher Nolan",NaN,NaN,"John David Washington, Robert Pattinson, Eliza...",Ludwig Göransson,Hoyte van Hoytema,Jennifer Lame,"Warner Bros. Pictures, Syncopy",Warner Bros. Pictures,"August 26, 2020 (2020-08-26) (United Kingdom),...",150 minutes,NaN,English,$200 million,$363.7 million
9,Soul,Pete Docter,Dana Murray,NaN,NaN,"Jamie Foxx, Tina Fey, Graham Norton, Rachel Ho...","Trent Reznor, Atticus Ross, Jon Batiste","Matt Aspbury, Ian Megibben",Kevin Nolting,"Walt Disney Pictures, Pixar Animation Studios",Walt Disney Studios Motion Pictures,"October 11, 2020 (2020-10-11) (BFI Fest), Dece...",100 minutes,United States,English,$150 million+,$119 million


,Name,Directed by,Produced by,Screenplay by,Based on,Starring,Music by,Cinematography,Edited by,Productioncompanies,Distributed by,Release date,Running time,Country,Language,Budget,Box office
0,Nomadland,Chloé Zhao,"Frances Mc Dormand, Peter Spears, Mollye Asher...",Chloé Zhao,Nomadland: Surviving America in the Twenty-Fir...,"Frances Mc Dormand, David Strathairn, Linda Ma...",Ludovico Einaudi,Joshua James Richards,Chloé Zhao,"Highwayman Films, Hear/Say Productions, Cor Co...",Searchlight Pictures,"September 11, 2020 (2020-09-11) (Venice), Febr...",108 minutes,United States,English,$5 million,$20.7 million
1,The Father,Florian Zeller,David Parfitt Jean-Louis Livi Philippe Carcass...,Florian Zeller Christopher Hampton,Le Pèreby Florian Zeller,"Anthony Hopkins, Olivia Colman, Mark Gatiss, I...",Ludovico Einaudi,Ben Smithard,Yorgos Lamprinos,"F comme Film, Trademark Films, Cine@, AG Studi...","Lionsgate(United Kingdom), UGC Distribution(Fr...","27 January 2020 (2020-01-27) (Sundance), 26 Ma...",97 minutes,NaN,English,$6 million,$14.1 million
2,Judas and the Black Messiah,Shaka King,"Charles D. King, Ryan Coogler, Shaka King","Will Berson, Shaka King",NaN,"Daniel Kaluuya, Lakeith Stanfield, Jesse Plemo...","Mark Isham, Craig Harris",Sean Bobbitt,Kristan Sprague,"MACRO, Participant, Bron Creative, Proximity",Warner Bros. Pictures,"February 12, 2021",126 minutes,United States,English,$26 million,$6.5 million
3,Minari,Lee Isaac Chung,"Dede Gardner, Jeremy Kleiner, Christina Oh",NaN,NaN,"Steven Yeun, Han Ye-ri, Alan Kim, Noel Kate Ch...",Emile Mosseri,Lachlan Milne,Harry Yoon,Plan B Entertainment,A24,"February 12, 2021",115 minutes,United States,Korean,$2 million,$15 million
4,Mank,David Fincher,"Ceán Chaffin, Eric Roth, Douglas Urbanski",Jack Fincher,NaN,"Gary Oldman, Amanda Seyfried, Lily Collins, Ar...","Trent Reznor, Atticus Ross",Erik Messerschmidt,Kirk Baxter,"Netflix International Pictures, Flying Studio",Netflix,"November 13, 2020 (2020-11-13) (United States)",131 minutes,United States,English,$25 million,"$122,252"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1269,"Toot, Whistle, Plunk and Boom",Ward Kimball Charles A. Nichols,Walt Disney,NaN,NaN,Bill Thompson Thurl Ravenscroft Loulie Jean N...,Joseph Dubin Sonny Burke (songs) Jack Elliot (...,NaN,NaN,Walt Disney Productions,Buena Vista Distribution,"November 10, 1953",10 mins (one reel),NaN,English,NaN,NaN
1270,Topkapi,Jules Dassin,Jules Dassin,NaN,The Light of Day1962 novelby Eric Ambler,Melina Mercouri Peter Ustinov Maximilian Schel...,Manos Hadjidakis,Henri Alekan,Roger Dwyre,Filmways,United Artists,"September 2, 1964 (1964-09-02)",120 min,United States,English,NaN,"$7,000,000"
1271,The Tortoise and the Hare,Wilfred Jackson,Walt Disney,NaN,NaN,Pinto Colvig Eddie Holden Marcellite Garner,Darrell Calker Frank Churchill,NaN,NaN,Walt Disney Productions,United Artists,"January 5, 1935 (1935-01-05)",8 minutes 17 seconds,NaN,English,NaN,NaN
1272,Torture Money,Harold S. Bucquet,Jack Chertok,NaN,NaN,Edwin Maxwell George Lynn,NaN,NaN,NaN,NaN,MGM,"January 2, 1937 (1937-01-02)",21 minutes,United States,English,NaN,NaN


In [95]:
movie_data_frame.fillna('Not Available')

,Name,Directed by,Produced by,Screenplay by,Based on,Starring,Music by,Cinematography,Edited by,Productioncompanies,Distributed by,Release date,Running time,Country,Language,Budget,Box office
0,Nomadland,Chloé Zhao,"Frances Mc Dormand, Peter Spears, Mollye Asher...",Chloé Zhao,Nomadland: Surviving America in the Twenty-Fir...,"Frances Mc Dormand, David Strathairn, Linda Ma...",Ludovico Einaudi,Joshua James Richards,Chloé Zhao,"Highwayman Films, Hear/Say Productions, Cor Co...",Searchlight Pictures,"September 11, 2020 (2020-09-11) (Venice), Febr...",108 minutes,United States,English,$5 million,$20.7 million
1,The Father,Florian Zeller,David Parfitt Jean-Louis Livi Philippe Carcass...,Florian Zeller Christopher Hampton,Le Pèreby Florian Zeller,"Anthony Hopkins, Olivia Colman, Mark Gatiss, I...",Ludovico Einaudi,Ben Smithard,Yorgos Lamprinos,"F comme Film, Trademark Films, Cine@, AG Studi...","Lionsgate(United Kingdom), UGC Distribution(Fr...","27 January 2020 (2020-01-27) (Sundance), 26 Ma...",97 minutes,Not Available,English,$6 million,$14.1 million
2,Judas and the Black Messiah,Shaka King,"Charles D. King, Ryan Coogler, Shaka King","Will Berson, Shaka King",Not Available,"Daniel Kaluuya, Lakeith Stanfield, Jesse Plemo...","Mark Isham, Craig Harris",Sean Bobbitt,Kristan Sprague,"MACRO, Participant, Bron Creative, Proximity",Warner Bros. Pictures,"February 12, 2021",126 minutes,United States,English,$26 million,$6.5 million
3,Minari,Lee Isaac Chung,"Dede Gardner, Jeremy Kleiner, Christina Oh",Not Available,Not Available,"Steven Yeun, Han Ye-ri, Alan Kim, Noel Kate Ch...",Emile Mosseri,Lachlan Milne,Harry Yoon,Plan B Entertainment,A24,"February 12, 2021",115 minutes,United States,Korean,$2 million,$15 million
4,Mank,David Fincher,"Ceán Chaffin, Eric Roth, Douglas Urbanski",Jack Fincher,Not Available,"Gary Oldman, Amanda Seyfried, Lily Collins, Ar...","Trent Reznor, Atticus Ross",Erik Messerschmidt,Kirk Baxter,"Netflix International Pictures, Flying Studio",Netflix,"November 13, 2020 (2020-11-13) (United States)",131 minutes,United States,English,$25 million,"$122,252"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1269,"Toot, Whistle, Plunk and Boom",Ward Kimball Charles A. Nichols,Walt Disney,Not Available,Not Available,Bill Thompson Thurl Ravenscroft Loulie Jean N...,Joseph Dubin Sonny Burke (songs) Jack Elliot (...,Not Available,Not Available,Walt Disney Productions,Buena Vista Distribution,"November 10, 1953",10 mins (one reel),Not Available,English,Not Available,Not Available
1270,Topkapi,Jules Dassin,Jules Dassin,Not Available,The Light of Day1962 novelby Eric Ambler,Melina Mercouri Peter Ustinov Maximilian Schel...,Manos Hadjidakis,Henri Alekan,Roger Dwyre,Filmways,United Artists,"September 2, 1964 (1964-09-02)",120 min,United States,English,Not Available,"$7,000,000"
1271,The Tortoise and the Hare,Wilfred Jackson,Walt Disney,Not Available,Not Available,Pinto Colvig Eddie Holden Marcellite Garner,Darrell Calker Frank Churchill,Not Available,Not Available,Walt Disney Productions,United Artists,"January 5, 1935 (1935-01-05)",8 minutes 17 seconds,Not Available,English,Not Available,Not Available
1272,Torture Money,Harold S. Bucquet,Jack Chertok,Not Available,Not Available,Edwin Maxwell George Lynn,Not Available,Not Available,Not Available,Not Available,MGM,"January 2, 1937 (1937-01-02)",21 minutes,United States,English,Not Available,Not Available
